In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies=pd.read_csv('TMDB_all_movies.csv', engine='python', on_bad_lines='warn')

/tmp/ipython-input-861410320.py:1: ParserWarning: Skipping line 18835: unexpected end of data

  movies=pd.read_csv('TMDB_all_movies.csv', engine='python', on_bad_lines='warn')


In [ ]:
movies=movies[['id' ,'title','overview','genres','cast','director','writers']]

In [ ]:
movies.dropna(inplace=True)


In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())
movies['genres']=movies['genres'].apply(lambda x:x.split(','))
movies['director']=movies['director'].apply(lambda x:x.split(','))
movies['writers']=movies['writers'].apply(lambda x:x.split(','))
movies['cast'] = movies['cast'].apply(lambda x: [c.strip() for c in x.split(',')[:3]])

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(' ','') for i in x])
movies['director']=movies['director'].apply(lambda x:[i.replace(' ','') for i in x])
movies['writers']=movies['writers'].apply(lambda x:[i.replace(' ','') for i in x])

In [ ]:
movies['tags'] = movies[['overview', 'cast', 'genres', 'director', 'writers']].apply(lambda x: sum([item if isinstance(item, list) else [] for item in x], []), axis=1)

Cleaning the new collected data


In [ ]:
new_df=movies[['id','title','tags']]

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

/tmp/ipython-input-487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

/tmp/ipython-input-4224080999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


Removing repeating or similar words


In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

/tmp/ipython-input-3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


Removing Stop Words


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words="english")

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
def recommend(movie):
  movie_index = new_df[new_df['title']==movie].index[0]
  distance= similarity[movie_index]
  movies_list=sorted(list(enumerate(distance)),reverse=True,key= lambda x:x[1][1:6])

  for i in movies_list:
    print(new_df.iloc[i[0]].title)
  return

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))